In [ ]:
def get_monthly_cost(reqs, req_mem, req_time, free_tier=True):
    free_reqs = 1000000
    million_reqs = 1000000
    cost_per_million_reqs = .20
    
    executions = (reqs - free_reqs) if free_tier else reqs
    
    req_costs = (executions / million_reqs) * cost_per_million_reqs if executions > 0 else 0
               
    compute_seconds = reqs * (req_time / 1000)
    compute_GB_seconds = compute_seconds * (req_mem / 1024)
    compute = (compute_GB_seconds - 400000) if free_tier else compute_GB_seconds
    
    cost_per_GB_seconds = 0.00001667
    compute_costs = compute * cost_per_GB_seconds if compute > 0 else 0
    
    return round(req_costs + compute_costs, 2)

In [ ]:
_million_reqs = 1000000

# Free tier, monthly freebies
_free_reqs = _million_reqs
_free_compute_GB_s = 400000

_remaining_free_hits = _free_reqs
_remaining_free_GB_s = _free_compute_GB_s

_cost_per_million_reqs = .20
_cost_per_GB_s = 0.00001667

_mem_per_req_MB = 128 
_time_per_req_ms = 500

def reset_free_tier_counters():
    global _remaining_free_hits
    _remaining_free_hits = _free_reqs
    
    global _remaining_free_GB_s 
    _remaining_free_GB_s = _free_compute_GB_s

def __get_free_tier_discount(resources, remaining_resources):
    return (0, remaining_resources-resources) if (remaining_resources > resources) \
        else (resources-remaining_resources, 0)
    
def get_hourly_cost(date, reqs):
    if date.day == 1 and date.hour == 0:
        reset_free_tier_counters()
    
    global _remaining_free_hits
    (reqs, _remaining_free_hits) = __get_free_tier_discount(reqs, _remaining_free_hits)

    compute_GB_s = reqs * _mem_per_req_MB/1024 * _time_per_req_ms/1000

    global _remaining_free_GB_s
    (compute_GB_s, _remaining_free_GB_s) = __get_free_tier_discount(compute_GB_s, _remaining_free_GB_s)
        
    return (reqs / _million_reqs) * _cost_per_million_reqs + compute_GB_s * _cost_per_GB_s